  
    
      
<a id='toc'></a>
<center><h1>Capstone - The Battle of Neighborhood</font></h1>
Segmenting and Clustering Neighborhoods of Sao Paulo and New York and comparing  
(Paulo Calabria)

----

## Table of Contents
- [I-Introduction](#introduction)
- [II-Objectives](#objective)
- [III-Data](#data)
- [VI-Methodology](#methodology)
    - [Analyze Sao Paulo](#analyzeKL)
    - [K-mean Cluster Sao Paulo](#kmeankl)
    - [Analyze New York](#analyzeJB)
    - [K-mean Cluster New York](#kmeanjb)
- [V-Results](#results)
- [VI-Discussion](#discussion)
- [VII-Conclusion](#conclusion)

<a id='introduction'></a>
# I-Introduction
This work is to provide the details of my final peer reviewed assignment for the IBM Data Science Professional Certificate program – Coursera Capstone. 
1.	A description of the problem and a discussion of the background - 
In the Americas there are many important cities that perform a leading role in the world economy. In particular we have the cities of New York, in the USA and the city of Sao Paulo in Brazil. Both cities have many things in common, like the fact that both are a melting pot of different cultures and both where build by many immigrants from various countries in Europe, Asia, Africa. In modern days, it would be helpful to have a way to compare the neighborhood of those cities to help the expatriates to chose places to go, to go for short of long assignment knowing in advance what they can encounter.
Research Question: What are similar neighborhoods I can find in Sao Paulo and New York City?



Brief information about both cities:
- São Paulo is a municipality in the Southeast Region of Brazil. The metropolis is an alpha global city (as listed by the GaWC) and the most populous city in Brazil, the Western Hemisphere and the Southern Hemisphere, besides being the largest Portuguese-speaking city in the world, and the largest city south of the Tropic of Capricorn. The municipality is also the world's 12th largest city proper by population. The city is the capital of the surrounding state of São Paulo, the most populous and wealthiest state in Brazil. It exerts strong international influences in commerce, finance, arts and entertainment. (source: https://en.wikipedia.org/wiki/São_Paulo)  

- New York (NY), is the most populous city in the United States. With an estimated 2018 population of 8,398,748 distributed over a land area of about 302.6 square miles (784 km2), New York is also the most densely populated major city in the United States.[10] Located at the southern tip of the state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass[11] and one of the world's most populous megacities,[12][13] with an estimated 19,979,477 people in its 2018 Metropolitan Statistical Area and 22,679,948 residents in its Combined Statistical Area.[3][4] A global power city,[14] New York City has been described as the cultural,[15][16][17][18][19] financial,[20][21] and media capital of the world,[22][23] and exerts a significant impact upon commerce,[21] entertainment, research, technology, education, politics, tourism, art, fashion, and sports. (source: https://en.wikipedia.org/wiki/New_York_City)

<a id='objective'></a>
# II-Objective
In Module 3, we explored New York City. In the Week 3, I have explored the city of Sao Paulo. For both we segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. I decided to adopt the interesting idea to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Sao Paulo? I think this is something that will be interesting for many travellers, business workers that need to go to those two Cities. In this project, we will study in details the area classification using Foursquare data and machine learning segmentation and clustering.

Using segmentation and clustering, the goal is to determine:
1. Similarity or dissimilarity of both cities.
2. Classification of area located inside each city and find similar clusters.
3. Generate data that can be used in the future for apps for the travelers

<a id='data'></a>
# III-Data
For New York, the data was acquired based on Module 3. For Sao Paulo, we got from wikipedia page and restructure to a xlsx file. This file will be later uploaded to my github. 


All the information about the venues were obtained based on the Foursquare data. Foursquare data were used for segmentation and clustering.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.




==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [2]:
#Sao Paulo Data processing
!pip install xlrd
Saopaulo_tarefa2 = "Saopaulo.TAREFA_2_df.xlsx"
Saopaulo_neighborhoods = pd.read_excel(Saopaulo_tarefa2)
Saopaulo_neighborhoods.shape

(40, 4)

In [3]:
Saopaulo_neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Água Rasa,Água Rasa,-23.5532,-46.5819
1,Alto de Pinheiros,Alto de Pinheiros,-23.5537,-46.7088
2,Anhanguera,Anhanguera,-23.4544,-46.7902
3,Aricanduva,Aricanduva,-23.5795,-46.5110
4,Artur Alvim‎,Artur Alvim‎,-23.5488,-46.4772


In [11]:
#NY Data processing - Getting data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [9]:
#NY Data processing continue
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()    

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [91]:
#Getting only Manhattan
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [92]:
manhattan_data.shape

(40, 4)

Data for both cities are now in neighborhoods (for NY City) and Saopaulo_neighborhoods (for Sao Paulo)

In [16]:
address = 'Sao Paulo, Brazil'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sao Paulo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sao Paulo are -23.5506507, -46.6333824.


In [17]:
# create map of Sao Paulo using latitude and longitude values
map_Saopaulo = folium.Map(location=[latitude, longitude], zoom_start=11)# add markers to map
for lat, lng, borough, in zip(Saopaulo_neighborhoods['Latitude'], Saopaulo_neighborhoods['Longitude'], Saopaulo_neighborhoods['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_Saopaulo)

Based on the data of Latitude and Longitude for both cities, we can now create maps with the neighborhoods marks

In [18]:
map_Saopaulo

In [86]:
from geopy.geocoders import Nominatim
import folium

nyaddress = 'Manhattan, NY'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(nyaddress)
nylatitude = location.latitude
nylongitude = location.longitude
print('The geograpical coordinate of Manhattan, NY are {}, {}.'.format(nylatitude, nylongitude))


The geograpical coordinate of Manhattan, NY are 40.7896239, -73.9598939.


In [95]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[nylatitude, nylongitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan


In [88]:
map_newyork

<a id='methodology'></a>
# VI-Methodology
I am going to use the same methodology as in Week 3 lab. Below are some considerations:  
- We already have a list of the principal boroughs/neighborhoods of the two cities (shown in the maps above).  
- We will use the Foursquare API to explore neighborhoods in both cities, New York and Sao Paulo
- Explore function to get the most common venue categories in each neighborhood, 
- Run a Clustering Algoritm to group the neighborhoods into clusters  

K-means clustering algorithm will be used. 
Folium library to visualize the neighborhoods in New York and Sao Paulo and their emerging clusters.  



Using Foursquare API to get venues at surounding area of both Sao Paulo and New York areas

#### Define Foursquare Credentials and Version

In [138]:
CLIENT_ID = 'My ID' # your Foursquare ID
CLIENT_SECRET = 'My Secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:


In [31]:
# The following function retrieves the venues given the names and coordinates and stores it into dataframe.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
#Sao Paulo
#Retrieve all venues given the Addresses 
#Saopaulo_neighborhoods = Saopaulo_data (not needed will use as bairros=Borough)
#Saopaulo_venues = getNearbyVenues(names=Saopaulo_neighborhoods['Neighbourhood'],
Saopaulo_venues = getNearbyVenues(names=Saopaulo_neighborhoods['Borough'],
                                   latitudes=Saopaulo_neighborhoods['Latitude'],
                                   longitudes=Saopaulo_neighborhoods['Longitude']
                                  )

Água Rasa
Alto de Pinheiros
Anhanguera
Aricanduva
Artur Alvim‎
Barra Funda
Bela Vista‎
Belém‎
Bom Retiro
Brasilândia‎
Butantã
Cachoeirinha
Cambuci
Campo Belo
Campo Grande
Campo Limpo
Cangaíba‎
Capão Redondo‎
Carrão
Casa Verde
Cidade Ademar‎
Cidade Dutra‎
Cidade Líder‎
Cidade Tiradentes
Consolação‎
Cursino‎
Ermelino Matarazzo
Freguesia do Ó‎
Grajaú‎
Guaianases‎
Iguatemi‎
Ipiranga‎
Itaim Bibi‎
Itaim Paulista‎
Itaquera‎
Jabaquara‎
Jaçanã‎
Jaguara‎
Jaguaré
Jaraguá


In [35]:
#dataframe Sao Paulo = Saopaulo_venues
print("Number of Venues for 40 bairros in Sao Paulo and #columns", Saopaulo_venues.shape)
Saopaulo_venues.sample(5)

Number of Venues for 40 bairros in Sao Paulo and #columns (717, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
366,Carrão,-23.5558,-46.5197,São Gabriel - Madeira de Demolição,-23.557436,-46.523435,Furniture / Home Store
232,Cachoeirinha,-23.4790,-46.6596,Academia Gaviões,-23.479204,-46.661649,Gym
697,Jaguara‎,-23.5072,-46.7618,Pães & Doces D'Abril,-23.506237,-46.763557,Bakery
140,Belém‎,-23.5471,-46.5912,大蝸里 花園,-23.548193,-46.595669,Garden
378,Carrão,-23.5558,-46.5197,Padaria Beirence,-23.558182,-46.516169,Bakery


In [36]:
#check how many venues were returned for each "Bairro" in Sao Paulo
print('There are {} uniques categories in Sao Paulo.'.format(len(Saopaulo_venues['Venue Category'].unique())))
Saopaulo_venues.groupby('Neighbourhood').count()

There are 174 uniques categories in Sao Paulo.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Alto de Pinheiros,20,20,20,20,20,20
Anhanguera,5,5,5,5,5,5
Aricanduva,7,7,7,7,7,7
Artur Alvim‎,9,9,9,9,9,9
Barra Funda,30,30,30,30,30,30
Bela Vista‎,30,30,30,30,30,30
Belém‎,29,29,29,29,29,29
Bom Retiro,30,30,30,30,30,30
Brasilândia‎,22,22,22,22,22,22


<a id='analyzeKL'></a>
# Analyze Sao Paulo                                                                        
<div style="text-align: right">[Top](#toc)</div>

In [38]:
#Analyze Each Borough Neighborhood
# one hot encoding
Saopaulo_onehot = pd.get_dummies(Saopaulo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Saopaulo_onehot['Neighbourhood'] = Saopaulo_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Saopaulo_onehot.columns[-1]] + list(Saopaulo_onehot.columns[:-1])
Saopaulo_onehot = Saopaulo_onehot[fixed_columns]

Saopaulo_onehot.sample(5)

,Neighbourhood,Acai House,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baiano Restaurant,Bakery,Bar,Beach,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Bookstore,Borek Place,Bowling Alley,Boxing Gym,Brazilian Restaurant,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Gym,College Stadium,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Korean Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Movie Theater,Museum,Music Venue,Nightclub,Northeastern Brazilian Restaurant,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southeastern Brazilian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapiocaria,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Plaza,Tour Provider,Toy / Game Store,Track,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Women's Store,Zoo Exhibit
183,Brasilândia‎,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,Belém‎,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
223,Butantã,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
569,Guaianases‎,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,Bela Vista‎,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [39]:
Saopaulo_onehot.shape

(717, 175)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [40]:
Saopaulo_grouped = Saopaulo_onehot.groupby('Neighbourhood').mean().reset_index()
Saopaulo_grouped

,Neighbourhood,Acai House,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baiano Restaurant,Bakery,Bar,Beach,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Bookstore,Borek Place,Bowling Alley,Boxing Gym,Brazilian Restaurant,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Gym,College Stadium,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Korean Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Movie Theater,Museum,Music Venue,Nightclub,Northeastern Brazilian Restaurant,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southeastern Brazilian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapiocaria,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Plaza,Tour Provider,Toy / Game Store,Track,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Women's Store,Zoo Exhibit
0,Alto de Pinheiros,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.05,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.050000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.050000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.00,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0

In [41]:
#Size:
Saopaulo_grouped.shape

(40, 175)

In [42]:
#pandas df
num_top_venues = 5
for neigh in Saopaulo_grouped['Neighbourhood']:
    print("----"+neigh+"----")
    temp = Saopaulo_grouped[Saopaulo_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alto de Pinheiros----
               venue  freq
0              Plaza  0.15
1  Convenience Store  0.10
2        Flea Market  0.05
3               Café  0.05
4        Coffee Shop  0.05


----Anhanguera----
                  venue  freq
0                   Bar   0.2
1            Restaurant   0.2
2            Toll Plaza   0.2
3  Brazilian Restaurant   0.2
4        Farmers Market   0.2


----Aricanduva----
                  venue  freq
0           Pizza Place  0.29
1            Pastelaria  0.14
2  Gym / Fitness Center  0.14
3              Pharmacy  0.14
4         Grocery Store  0.14


----Artur Alvim‎----
            venue  freq
0    Burger Joint  0.11
1  Soccer Stadium  0.11
2      Boxing Gym  0.11
3    Soccer Field  0.11
4         Stadium  0.11


----Barra Funda----
                 venue  freq
0            Nightclub  0.20
1              Theater  0.10
2                  Pub  0.07
3               Museum  0.07
4  Indie Movie Theater  0.03


----Bela Vista‎----
                venue  fr

In [43]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
spneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
spneighborhoods_venues_sorted['Neighbourhood'] = Saopaulo_grouped['Neighbourhood']

for ind in np.arange(Saopaulo_grouped.shape[0]):
    spneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Saopaulo_grouped.iloc[ind, :], num_top_venues)

spneighborhoods_venues_sorted.shape

(40, 11)

In [54]:
spneighborhoods_venues_sorted.sample(5)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Cidade Tiradentes,Snack Place,BBQ Joint,Flea Market,Fast Food Restaurant,Bus Station,Bakery,Pizza Place,Theater,Gym / Fitness Center,Dessert Shop
21,Cidade Líder‎,Bakery,Dessert Shop,Gym / Fitness Center,Convenience Store,Comfort Food Restaurant,Restaurant,Brazilian Restaurant,Market,Food Truck,Liquor Store
24,Cursino‎,Bakery,Acai House,Fruit & Vegetable Store,Bar,Pizza Place,Burger Joint,Middle Eastern Restaurant,Furniture / Home Store,Flower Shop,Medical Center
39,Água Rasa,Farmers Market,Brazilian Restaurant,Bakery,Gym,Sushi Restaurant,Pharmacy,Pastelaria,Convenience Store,Deli / Bodega,Diner
3,Artur Alvim‎,Soccer Stadium,Boxing Gym,Grocery Store,Stadium,Tour Provider,Burger Joint,Clothing Store,Farmers Market,Soccer Field,Event Space


<a id='kmeankl'></a>
# K-mean Cluster Sao Paulo
<div style="text-align: right">[Top](#toc)</div>

In [55]:
#Run k-means to cluster the neighborhood into 10 clusters.
# set number of clusters
kclusters = 10

Saopaulo_grouped_clustering = Saopaulo_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Saopaulo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[3 4 1 3 3 3 3 3 3 3]
40


In [56]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Saopaulo_merged = Saopaulo_neighborhoods

# add clustering labels
spneighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#Saopaulo_merged['Cluster Labels'] = kmeans.labels_

# merge Saopaulo_grouped with toronto_data to add latitude/longitude for each neighborhood
Saopaulo_merged = Saopaulo_merged.join(spneighborhoods_venues_sorted.set_index('Neighbourhood'), on='Borough')
# check columns!
Saopaulo_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Água Rasa,Água Rasa,-23.5532,-46.5819,3,Farmers Market,Brazilian Restaurant,Bakery,Gym,Sushi Restaurant,Pharmacy,Pastelaria,Convenience Store,Deli / Bodega,Diner
1,Alto de Pinheiros,Alto de Pinheiros,-23.5537,-46.7088,3,Plaza,Convenience Store,Bar,Restaurant,Coffee Shop,Café,Business Service,Fast Food Restaurant,Flea Market,Food Truck
2,Anhanguera,Anhanguera,-23.4544,-46.7902,4,Brazilian Restaurant,Farmers Market,Restaurant,Bar,Toll Plaza,Zoo Exhibit,Electronics Store,Flower Shop,Flea Market,Fish Market
3,Aricanduva,Aricanduva,-23.5795,-46.5110,1,Pizza Place,Bakery,Gym / Fitness Center,Grocery Store,Pharmacy,Pastelaria,Fish Market,Fast Food Restaurant,Farmers Market,Exhibit
4,Artur Alvim‎,Artur Alvim‎,-23.5488,-46.4772,3,Soccer Stadium,Boxing Gym,Grocery Store,Stadium,Tour Provider,Burger Joint,Clothing Store,Farmers Market,Soccer Field,Event Space


In [57]:
Saopaulo_merged.sample(5)

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Freguesia do Ó‎,Freguesia do Ó‎,-23.5015,-46.6977,3,Burger Joint,Pizza Place,Bar,Acai House,Tapiocaria,Chocolate Shop,Martial Arts Dojo,Church,Brewery,Snack Place
15,Campo Limpo,Campo Limpo,-23.6348,-46.7550,1,Food Truck,Plaza,Dumpling Restaurant,Gym / Fitness Center,Pizza Place,Construction & Landscaping,Dessert Shop,Park,Coffee Shop,Clothing Store
17,Capão Redondo‎,Capão Redondo‎,-23.6685,-46.7690,1,Bakery,Gym,Pizza Place,Plaza,Fast Food Restaurant,Convenience Store,Sandwich Place,Brazilian Restaurant,Soup Place,Health & Beauty Service
5,Barra Funda,Barra Funda,-23.5271,-46.6597,3,Nightclub,Theater,Pub,Museum,Korean Restaurant,Martial Arts Dojo,Lounge,Café,Farmers Market,Exhibit
21,Cidade Dutra‎,Cidade Dutra‎,-23.7117,-46.7039,3,Dessert Shop,Bar,Brazilian Restaurant,Pharmacy,Department Store,Burger Joint,Plaza,Pet Store,Pizza Place,Steakhouse


In [58]:
#Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
#for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],kmeans.labels_):
for lat, lon, poi, cluster in zip(Saopaulo_neighborhoods['Latitude'], Saopaulo_neighborhoods['Longitude'], Saopaulo_neighborhoods['Borough'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='results'></a>
# Results for Sao Paulo                                                                         
<div style="text-align: right">[Top](#toc)</div>

In [67]:
#Cluster 1 for Sao Paulo (Has 1 Bairro)
Saopaulo_merged.loc[Saopaulo_merged['Cluster Labels'] == 0, Saopaulo_merged.columns[[1] + list(range(4, Saopaulo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Grajaú‎,0,Business Service,Zoo Exhibit,Cosmetics Shop,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Exhibit


In [69]:
#Cluster 2 for Sao Paulo  (Has 11 Bairros)
Saopaulo_merged.loc[Saopaulo_merged['Cluster Labels'] == 1, Saopaulo_merged.columns[[1] + list(range(4, Saopaulo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Aricanduva,1,Pizza Place,Bakery,Gym / Fitness Center,Grocery Store,Pharmacy,Pastelaria,Fish Market,Fast Food Restaurant,Farmers Market,Exhibit
11,Cachoeirinha,1,Pizza Place,Food Truck,Bakery,Soccer Field,Shoe Store,Food Stand,Steakhouse,Supermarket,Dumpling Restaurant,Plaza
15,Campo Limpo,1,Food Truck,Plaza,Dumpling Restaurant,Gym / Fitness Center,Pizza Place,Construction & Landscaping,Dessert Shop,Park,Coffee Shop,Clothing Store
17,Capão Redondo‎,1,Bakery,Gym,Pizza Place,Plaza,Fast Food Restaurant,Convenience Store,Sandwich Place,Brazilian Restaurant,Soup Place,Health & Beauty Service
18,Carrão,1,Gym / Fitness Center,Bus Station,Juice Bar,Dessert Shop,Comfort Food Restaurant,Chocolate Shop,Sandwich Place,Fast Food Restaurant,Smoke Shop,Flower Shop
22,Cidade Líder‎,1,Bakery,Dessert Shop,Gym / Fitness Center,Convenience Store,Comfort Food Restaurant,Restaurant,Brazilian Restaurant,Market,Food Truck,Liquor Store
23,Cidade Tiradentes,1,Snack Place,BBQ Joint,Flea Market,Fast Food Restaurant,Bus Station,Bakery,Pizza Place,Theater,Gym / Fitness Center,Dessert Shop
29,Guaianases‎,1,Gym / Fitness Center,Food Truck,Pharmacy,Department Store,Shoe Store,Zoo Exhibit,Electronics Store,Flea Market,Fish Market,Fast Food Restaurant
33,Itaim Paulista‎,1,Dessert Shop,Bowling Alley,Japanese Restaurant,Pizza Place,Fast Food Restaurant,Shipping Store,Brewery,Brazilian Restaurant,Pharmacy,Food Truck
35,Jabaquara‎,1,Brazilian Restaurant,Japanese Restaurant,Park,Café,History Museum,Sushi Restaurant,Gym,Gym / Fitness Center,Hobby Shop,Massage Studio


In [70]:
#Cluster 3 for Sao Paulo  (Has 1 Bairro)
Saopaulo_merged.loc[Saopaulo_merged['Cluster Labels'] == 2, Saopaulo_merged.columns[[1] + list(range(4, Saopaulo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Jaraguá,2,IT Services,Zoo Exhibit,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Exhibit


In [71]:
#Cluster 4 for Sao Paulo  (Has 21 Bairro)
Saopaulo_merged.loc[Saopaulo_merged['Cluster Labels'] == 3, Saopaulo_merged.columns[[1] + list(range(4, Saopaulo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Água Rasa,3,Farmers Market,Brazilian Restaurant,Bakery,Gym,Sushi Restaurant,Pharmacy,Pastelaria,Convenience Store,Deli / Bodega,Diner
1,Alto de Pinheiros,3,Plaza,Convenience Store,Bar,Restaurant,Coffee Shop,Café,Business Service,Fast Food Restaurant,Flea Market,Food Truck
4,Artur Alvim‎,3,Soccer Stadium,Boxing Gym,Grocery Store,Stadium,Tour Provider,Burger Joint,Clothing Store,Farmers Market,Soccer Field,Event Space
5,Barra Funda,3,Nightclub,Theater,Pub,Museum,Korean Restaurant,Martial Arts Dojo,Lounge,Café,Farmers Market,Exhibit
6,Bela Vista‎,3,Bar,Antique Shop,Brazilian Restaurant,Gymnastics Gym,Italian Restaurant,Pizza Place,Plaza,Café,Coffee Shop,Jazz Club
7,Belém‎,3,Bar,Burger Joint,Restaurant,Gym,Ice Cream Shop,Market,Brazilian Restaurant,Chocolate Shop,Coffee Shop,Italian Restaurant
8,Bom Retiro,3,Women's Store,Café,Korean Restaurant,Diner,Pizza Place,Pastelaria,Chocolate Shop,Restaurant,Coffee Shop,Pharmacy
9,Brasilândia‎,3,Gym,Food Truck,Department Store,Dessert Shop,Convenience Store,Pizza Place,Diner,Comfort Food Restaurant,Recording Studio,Restaurant
10,Butantã,3,Science Museum,Track,Bus Station,Hot Dog Joint,Zoo Exhibit,Bakery,College Gym,College Stadium,Food Truck,Gym / Fitness Center
12,Cambuci,3,Brazilian Restaurant,Bakery,Restaurant,Market,Burger Joint,Sandwich Place,Chocolate Shop,Clothing Store,Food & Drink Shop,Spa


Cluters 2 and 4 have the largest number of "Bairros"

<a id='analyzeJB'></a>
# Analyze NY City                                                                       
<div style="text-align: right">[Top](#toc)</div>

In [89]:
print("Number of neighborhoods in New York and #columns", neighborhoods.shape)
neighborhoods.head()

Number of neighborhoods in New York and #columns (306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [83]:
#New York
#Retrieve all venues given the Addresses 
#Saopaulo_venues = getNearbyVenues(names=Saopaulo_neighborhoods['Neighborhood'],
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [84]:
#dataframe New York = ny_venues
print("Number of Venues for xx Boroughs in Sao Paulo and #columns", ny_venues.shape)
ny_venues.sample(5)

Number of Venues for xx Boroughs in Sao Paulo and #columns (6114, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4155,Hillcrest,40.723825,-73.797603,The UPS Store,40.725784,-73.792534,Shipping Store
5536,Turtle Bay,40.752042,-73.967708,I Love Kickboxing - 43rd Street,40.750580,-73.971540,Martial Arts Dojo
262,Bedford Park,40.870185,-73.885512,Dunkin',40.872716,-73.886468,Donut Shop
4635,Woodrow,40.541968,-74.205246,Chase Bank,40.542043,-74.207331,Bank
5305,Mill Basin,40.615974,-73.915154,Dagan's Kosher Pizza,40.613680,-73.912247,Pizza Place


In [97]:
#Changing to analyze Manhathan only
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [98]:
print(manhattan_venues.shape)
manhattan_venues.head()

(1183, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [100]:
manhattan_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Battery Park City,30,30,30,30,30,30
Carnegie Hill,30,30,30,30,30,30
Central Harlem,30,30,30,30,30,30
Chelsea,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30
Civic Center,30,30,30,30,30,30
Clinton,30,30,30,30,30,30
East Harlem,30,30,30,30,30,30
East Village,30,30,30,30,30,30


Let's find out how many unique categories can be curated from all the returned venues¶

In [101]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 239 uniques categories.


## Analyze Each Neighborhood of Manhattan

In [104]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighbourhood'] = manhattan_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Filipino Restaurant,Financial or Legal Service,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1

In [105]:
manhattan_onehot.shape

(1183, 240)

In [107]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶
manhattan_grouped = manhattan_onehot.groupby('Neighbourhood').mean().reset_index()
manhattan_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Filipino Restaurant,Financial or Legal Service,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000

In [108]:
#Let's confirm the new size¶
manhattan_grouped.shape

(40, 240)

In [109]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0   Memorial Site  0.10
1            Park  0.10
2  Sandwich Place  0.07
3     Coffee Shop  0.07
4      Food Court  0.07


----Carnegie Hill----
                  venue  freq
0           Pizza Place  0.07
1  Gym / Fitness Center  0.07
2                   Gym  0.07
3    Italian Restaurant  0.07
4             Bookstore  0.07


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1  American Restaurant  0.07
2                  Bar  0.07
3    French Restaurant  0.07
4            Juice Bar  0.03


----Chelsea----
                venue  freq
0               Hotel  0.10
1           Nightclub  0.07
2  Italian Restaurant  0.07
3             Theater  0.07
4   French Restaurant  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.10
1                 Spa  0.07
2        Noodle House  0.07
3      Sandwich Place  0.07
4         Pizza Place  0.03


----Civic Center----
                  venue  freq


In [110]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [122]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = manhattan_grouped['Neighbourhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Memorial Site,Coffee Shop,Sandwich Place,Food Court,Building,Shopping Mall,Sushi Restaurant,Burrito Place,Smoke Shop
1,Carnegie Hill,Gym,Bookstore,Coffee Shop,Pizza Place,Italian Restaurant,Gym / Fitness Center,Community Center,Café,French Restaurant,Spa
2,Central Harlem,French Restaurant,African Restaurant,American Restaurant,Bar,Cosmetics Shop,Bookstore,Beer Bar,Gym / Fitness Center,Library,Music Venue
3,Chelsea,Hotel,Italian Restaurant,Nightclub,Theater,Indian Restaurant,Chinese Restaurant,Beer Bar,Speakeasy,Café,Seafood Restaurant
4,Chinatown,Chinese Restaurant,Spa,Sandwich Place,Noodle House,Spanish Restaurant,Bubble Tea Shop,Salon / Barbershop,Roof Deck,Cocktail Bar,New American Restaurant


<a id='kmeanjb'></a>
# K-mean Cluster Manhattan
<div style="text-align: right">[Top](#toc)</div>

In [123]:
#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 2, 4, 0, 0])

In [124]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [125]:
neighborhoods_venues_sorted.sample()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,0,East Village,Pizza Place,Wine Bar,Dessert Shop,Vietnamese Restaurant,Coffee Shop,Speakeasy,Swiss Restaurant,Juice Bar,Beer Store,Gift Shop


In [126]:
manhattan_data.sample()

,Borough,Neighborhood,Latitude,Longitude
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [127]:
manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

manhattan_merged.head() # check the head columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Sandwich Place,Coffee Shop,American Restaurant,Donut Shop,Shopping Mall,Seafood Restaurant,Steakhouse,Supplement Shop,Clothing Store,Kids Store
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Spa,Sandwich Place,Noodle House,Spanish Restaurant,Bubble Tea Shop,Salon / Barbershop,Roof Deck,Cocktail Bar,New American Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Wine Shop,Bakery,Park,Indian Restaurant,Pool,Caribbean Restaurant,Scenic Lookout,Cocktail Bar,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Frozen Yogurt Shop,Park,Café,Bakery,American Restaurant,Deli / Bodega,Wine Bar,Diner,Dog Run
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Coffee Shop,Yoga Studio,Café,Caribbean Restaurant,Cocktail Bar,Mexican Restaurant,Japanese Restaurant,School,Pizza Place,Park


In [129]:
#Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[nylatitude, nylongitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='results'></a>
# Results for Manhattan                                                                        
<div style="text-align: right">[Top](#toc)</div>

#### Cluster 1 (Manhattan)

In [131]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,American Restaurant,Donut Shop,Shopping Mall,Seafood Restaurant,Steakhouse,Supplement Shop,Clothing Store,Kids Store
1,Chinatown,Chinese Restaurant,Spa,Sandwich Place,Noodle House,Spanish Restaurant,Bubble Tea Shop,Salon / Barbershop,Roof Deck,Cocktail Bar,New American Restaurant
5,Manhattanville,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Coffee Shop,Indian Restaurant,Diner,Bike Trail,Boutique,Spanish Restaurant,Café
6,Central Harlem,French Restaurant,African Restaurant,American Restaurant,Bar,Cosmetics Shop,Bookstore,Beer Bar,Gym / Fitness Center,Library,Music Venue
8,Upper East Side,Hotel,American Restaurant,Italian Restaurant,Park,Boutique,Sculpture Garden,Sushi Restaurant,Exhibit,Coffee Shop,Bookstore
12,Upper West Side,Bakery,Wine Bar,Italian Restaurant,American Restaurant,Café,Greek Restaurant,Juice Bar,Gift Shop,Bookstore,Lingerie Store
15,Midtown,Park,Hotel,Tailor Shop,Chinese Restaurant,Sporting Goods Shop,Bookstore,Food Truck,Spa,Smoke Shop,Skating Rink
17,Chelsea,Hotel,Italian Restaurant,Nightclub,Theater,Indian Restaurant,Chinese Restaurant,Beer Bar,Speakeasy,Café,Seafood Restaurant
19,East Village,Pizza Place,Wine Bar,Dessert Shop,Vietnamese Restaurant,Coffee Shop,Speakeasy,Swiss Restaurant,Juice Bar,Beer Store,Gift Shop
21,Tribeca,American Restaurant,Wine Shop,Spa,Park,Yoga Studio,Hotel,Salad Place,Poke Place,Playground,Men's Store


#### Cluster 2 (Manhattan)

In [132]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Coffee Shop,Yoga Studio,Café,Caribbean Restaurant,Cocktail Bar,Mexican Restaurant,Japanese Restaurant,School,Pizza Place,Park
16,Murray Hill,Japanese Restaurant,Coffee Shop,Hotel,Burger Joint,Italian Restaurant,Tea Room,Restaurant,Ramen Restaurant,Lounge,Speakeasy
18,Greenwich Village,Italian Restaurant,Café,French Restaurant,Sushi Restaurant,Clothing Store,Cosmetics Shop,Coffee Shop,Cocktail Bar,Salon / Barbershop,Beer Bar
20,Lower East Side,Café,Japanese Restaurant,Ramen Restaurant,Coffee Shop,Cocktail Bar,Art Gallery,Yoga Studio,Speakeasy,Pet Café,Mediterranean Restaurant
22,Little Italy,Café,Sandwich Place,Optical Shop,Ice Cream Shop,Wine Bar,Chinese Restaurant,French Restaurant,Spanish Restaurant,Martial Arts Dojo,Karaoke Bar
23,Soho,Women's Store,Clothing Store,Men's Store,Shoe Store,Boutique,Salon / Barbershop,Optical Shop,Ski Shop,French Restaurant,Miscellaneous Shop
24,West Village,Italian Restaurant,Cosmetics Shop,Cocktail Bar,Bakery,Chinese Restaurant,Gastropub,Gourmet Shop,Coffee Shop,Accessories Store,French Restaurant
31,Noho,French Restaurant,Coffee Shop,Wine Shop,Ice Cream Shop,Boutique,Cocktail Bar,Italian Restaurant,Rock Club,Hotel,Grocery Store


#### Cluster 3 (Manhattan)

In [135]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Theater,Indie Movie Theater,Performing Arts Venue,Concert Hall,Plaza,Gym / Fitness Center,Food Truck,Fountain,Circus,Gift Shop
14,Clinton,Gym / Fitness Center,Theater,Hotel,Pizza Place,Café,Sporting Goods Shop,Lounge,French Restaurant,Mediterranean Restaurant,Building


#### Cluster 4 (Manhattan)

In [136]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midtown South,Korean Restaurant,Hotel,Cosmetics Shop,Coffee Shop,Fried Chicken Joint,Lingerie Store,Clothing Store,Scenic Lookout,Snack Place,Building


#### Cluster 5 (Manhattan)

In [137]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Wine Shop,Bakery,Park,Indian Restaurant,Pool,Caribbean Restaurant,Scenic Lookout,Cocktail Bar,Coffee Shop
3,Inwood,Mexican Restaurant,Frozen Yogurt Shop,Park,Café,Bakery,American Restaurant,Deli / Bodega,Wine Bar,Diner,Dog Run
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Pizza Place,Taco Place,Street Art,Steakhouse,Spanish Restaurant,French Restaurant
9,Yorkville,Deli / Bodega,Wine Shop,Park,Coffee Shop,Italian Restaurant,Liquor Store,Asian Restaurant,Monument / Landmark,Sandwich Place,Beer Store
10,Lenox Hill,Gym,Wine Shop,Burger Joint,Gym / Fitness Center,Thai Restaurant,Japanese Restaurant,Smoke Shop,French Restaurant,Salad Place,Liquor Store
11,Roosevelt Island,Coffee Shop,Park,Sandwich Place,Japanese Restaurant,Bus Line,Metro Station,Soccer Field,School,Scenic Lookout,Liquor Store
25,Manhattan Valley,Pizza Place,Bar,Coffee Shop,Yoga Studio,Hawaiian Restaurant,Park,Noodle House,Mexican Restaurant,Korean Restaurant,Italian Restaurant
36,Tudor City,Park,Pizza Place,Mexican Restaurant,Deli / Bodega,Thai Restaurant,Restaurant,Spanish Restaurant,Bridge,Burger Joint,Café
37,Stuyvesant Town,Boat or Ferry,Bar,Park,Baseball Field,Harbor / Marina,Gas Station,Farmers Market,German Restaurant,Cocktail Bar,Coffee Shop


<a id='discussion'></a>
# VI-Discussion                                                                         
<div style="text-align: right">[Top](#toc)</div>

With the data on the clusters for each cities above, we can have an idea of each one and the type of venues categories (most common) would you find in each cities. Our original idea is to use this data to help the expatriates to chose places to go, to go for short of long assignment, based on the similarities of the type of Neighborhood they will find.

We can certainly say that Sao Paulo has many "Bairros" (Neighborhoods) that are very similar to what you would find in Manhattan and the data helps to identify them  

In general, with the initial data obtained for this project we can make the following assumptions for each cluster:
If you are looking for places to stay and have many alternatives nearby:
- Cluster 2 and 5 in Manhattan 
- Cluster 2 and 4 in Sao Paulo

One idea to enhance this initial project would be to go deeper in the correlation of the Neighborhood by finding a quantitative way to identify and distinguish their difference based on most common venues founded via Foursquare. 
Another idea, is to design an App where the user could give her/his most important venue that she or he would like to find in the target Neighbourhood at the city and run this analysis and come back with possible suggestion. I think this would be very helpful and can be done in a future project.

I believe that this project is a first step towards a quantitative and systematic comparison of the different cities, and specially that is bring information about the city of Sao Paulo, which I could not find any similar work done before. 

<a id='conclusion'></a>
# VII-Conclusion                                                                         
<div style="text-align: right">[Top](#toc)</div>


Although I think the goals of this project were met ( Applied Data Science Capstone Project) there is definitely room for further improvement and development as discussed above.

My initial experience as a Data Scientist trying to do this project and others, is that the time you spent in getting the right data to answer your initial "Research Question" represents the majority of the time spent.
The right definition of the problem to be solved is key to make sure you go after the right data needed.
In this aspects I may say from my experience that the Data Science work and methodology is very similar to scientific methodology, and the risks associated with finding out that your objective is too aggressive and you will need more analysis / data to accomplish are present. 


Paulo Cesar Calabria
    
Sao Paulo, SP - Brazil
pcalabria@fei.edu.br